На основании представленных ссылок на спецификации различных моделей тракторов сделайте нейро-консультанта для ответов на вопросы покупателя, используя динамический выбор из разных баз знаний.

Ссылки для создания БЗ:

https://ventrac.com.ua/uploads/series-4000/spec/spec/4500%20%D0%BE%D0%B1%D1%89%D0%B8%D0%B9%20%D1%84%D0%BB%D0%B0%D0%B5%D1%80_1.compressed.pdf


http://agromester.md/pdf/9r.pdf

Проверьте работу нейро-консультанта на самостоятельно сформулированных вопросах.

In [ ]:
!pip install tiktoken==0.4.0 openai==0.28.0 langchain==0.0.281 unstructured pdf2image pdfminer.six faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 63.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:

In [ ]:
import tiktoken
import getpass
import openai
import tiktoken
from openai.embeddings_utils import distances_from_embeddings
import pandas as pd
import numpy as np
from ast import literal_eval
from langchain.chains.router import MultiRetrievalQAChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import os
from openai.embeddings_utils import distances_from_embeddings, cosine_similarity
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# Скачиваем файл и сохраняет его
def download_pdf_from_url(url, save_path):
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded and saved as {save_path}")
    else:
        print(f"Не удалось загрузить PDF. Код состояния HTTP: {response.status_code}")

url = "https://ventrac.com.ua/uploads/series-4000/spec/spec/4500%20%D0%BE%D0%B1%D1%89%D0%B8%D0%B9%20%D1%84%D0%BB%D0%B0%D0%B5%D1%80_1.compressed.pdf"
save_path = "/content/Ventrac.pdf"

download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/Ventrac.pdf


In [ ]:
url = "https://agromester.md/pdf/9r.pdf"
save_path = "/content/JohnDeere.pdf"

download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/JohnDeere.pdf


In [ ]:
# функция, которая принимает список названий PDF-файлов и возвращает базу знаний FAISS
def create_faiss_vectorstore(pdf_names):
    pdf_folder_path = "/content/"

    # Фильтрация файлов: загрузка только файлов с определенными именами и расширением .pdf
    all_files = os.listdir(pdf_folder_path)
    pdf_files = [fn for fn in all_files if fn in pdf_names and fn.endswith('.pdf')]

    # Создание объектов UnstructuredPDFLoader для каждого PDF-файла
    loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in pdf_files]

    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, length_function=len)

    for loader in loaders:
        # Загрузка и разделение текста
        raw_documents = loader.load()
        documents = text_splitter.split_documents(raw_documents)
        all_documents.extend(documents)

    # Создание FAISS vectorstore
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(all_documents, OpenAIEmbeddings()).as_retriever()

    return vectorstore

In [ ]:
# Создаем БЗ по каждой спецификации (pdf)
pdf_name_Ventrac = ['Ventrac.pdf']
db_Ventrac = create_faiss_vectorstore(pdf_name_Ventrac)
pdf_name_JohnDeere = ['JohnDeere.pdf']
db_JohnDeere = create_faiss_vectorstore(pdf_name_JohnDeere)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Добавляем информацию и описание для векторных хранилищ (баз знаний)
retriever_infos = [
    {
        "name": "Traktor JohnDeere 9R/RT",
        "description": "Применяется для ответов на вопросы по модели трактора John Deere серии 9R.",  # описываем, для каких вопросов применять данную БЗ
        "retriever": db_JohnDeere
    },
    {
        "name": "Traktor Ventrac",
        "description": "Применяется для ответов на вопросы по модели трактора Ventrac серии 4500", # описываем, для каких вопросов применять данную БЗ
        "retriever": db_Ventrac
    }
]

In [ ]:
# Иициализируем класс
chain = MultiRetrievalQAChain.from_retrievers(ChatOpenAI(temperature=0), retriever_infos, verbose=True)

In [ ]:
print(chain.run("Что может убирать трактор Ventrac?"))



> Entering new MultiRetrievalQAChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Traktor Ventrac: {'query': 'Что может убирать трактор Ventrac?'}
> Finished chain.
Трактор Ventrac серии 4500 может выполнять широкий спектр работ, связанных с уборкой и обработкой различных поверхностей. Он может убирать территории, ухаживать за газоном и почвой, а также убирать снег. Благодаря более чем 30-ти видам приспособлений и насадок, Ventrac 4500 обладает абсолютной универсальностью и может выполнять работы, связанные со строительством, уборкой территорий и многое другое.


In [ ]:
print(chain.run("Какие типы двигателя устанавливаются на трактор?"))



> Entering new MultiRetrievalQAChain chain...
None: {'query': 'Какие типы двигателя устанавливаются на трактор?'}
> Finished chain.
На тракторах могут быть установлены различные типы двигателей, в зависимости от модели и производителя. Некоторые из наиболее распространенных типов двигателей, используемых на тракторах, включают дизельные двигатели, бензиновые двигатели и газовые двигатели. Дизельные двигатели обычно предпочитаются из-за своей высокой мощности и эффективности, а также возможности использования различных видов топлива. Бензиновые двигатели могут быть более доступными в некоторых регионах и иметь свои преимущества в небольших тракторах. Газовые двигатели, такие как пропан или природный газ, также могут использоваться на некоторых тракторах для экономии топлива и снижения выбросов. Конкретные типы и модели двигателей будут зависеть от конкретной марки и модели трактора.


In [ ]:
print(chain.run("Какой двигатель может работать на тракторе Ventrac при 30 градусах уклона непрерывно?"))



> Entering new MultiRetrievalQAChain chain...
Traktor Ventrac: {'query': 'Какой двигатель может работать на тракторе Ventrac при 30 градусах уклона непрерывно?'}
> Finished chain.
Двигатель Кавасаки DFI на тракторе Ventrac серии 4500 может работать при 30 градусах уклона непрерывно.
